# Write an IP problem in <'lp'> format 

In [1]:
%load_ext autoreload
%autoreload 2
#! pip install pulp
## https://towardsdatascience.com/linear-programming-using-python-priyansh-22b5ee888fe0

In [2]:
# To be a real programmer : get a black screen 
from jupyterthemes import get_themes
import jupyterthemes as jt
from jupyterthemes.stylefx import set_nb_theme
set_nb_theme('chesterish')

In [3]:
from pulp import *
import pandas as pd 
import numpy as np 


In [4]:
# Import files 
adjacency = np.loadtxt('Adjacency.txt', delimiter = ",")
parcels = np.loadtxt('Parcels.txt', delimiter = ",")
adjacency = adjacency.astype(int)


In [5]:
# Data definition 

n_parcels = 120 
B = 1000000
m = 121 #Big Number !! just need to be big enough 

# Parcels 
parcel = parcels[:,0]

# Cost Matrix 
cost_matrix = parcels[:,2]

# Area Matrix 
area_matrix = parcels[:,1]

# Parcels i adjacent to j 
adj1 = adjacency[:,0]

# Parcels j adjacent to i 
adj2 = adjacency[:,1]

In [6]:
# Model initialization
model= LpProblem("Lab4", LpMaximize)

In [7]:
# Decision variables 
variable_names = [str(i) for i in range(1,n_parcels+1)]

DV_variables = LpVariable.matrix("X", variable_names, cat = "Binary", lowBound= 0 )
X_allocation = np.array(DV_variables).reshape(120)

# Flow variables
adj_variable_names = [str(adj1[i]) + '_' + str(adj2[j]) for i in range (len(adj1)) for j in range (len(adj2))]
Flow_variables = LpVariable.matrix("Y", adj_variable_names, cat = "Binary", lowBound = 0)
Yij_allocation = np.array(Flow_variables).reshape(260,260)

#Tail variables 

Zij_variable_names = [str(adj1[i]) + '_' + str(adj2[i]) for i in range (len(adj1))]
Zij_variables = LpVariable.matrix("Z", Zij_variable_names, cat = "Integer", lowBound = 0)
Zij_allocation = np.array(Zij_variables).reshape(260)

W_variable_names = [str(i) for i in range(1,n_parcels+1)]
W_variables = LpVariable.matrix("W", W_variable_names, cat = "Integer", lowBound= 0 )
W_allocation = np.array(W_variables).reshape(120)



In [8]:
# Constraints 

## Constraint 1: Budget constraint 
### SUM pi*Xi <= B 

## Constraint 2: 4 adjacent cells 
### SUM Yij (i e Ij)<= [|hdhdhd]*Xj for all j 

## Constraint 3: Max 1 connection from i
#### SUM Yij (j e Ii) <= Xi 

## Constraint 4: Number of arcs must be less or equal to the number of nodes 
#### SUM Yij (i,j) <= SUM Xi - 1 (i)

## Constraint 5: tail function 
### Zij >= Wi + 1 - m(1-Yij) for all i, j adjacent, i != j 

## Constraint 6: cycle breaking constraint 
### Wj >= SUM (ieIj) Zij for all j 

## Constraint 7: ensure that zij is minimal 
### Zij <= m*Yij 

## Constraint 8:ensure that zij is minimal
### Zij <= Wi+1 

#core parcel :  parcel 10, 32.031, 0.00

In [9]:
## Constraint 1: Budget constraint 
### SUM pi*Xi <= B 

model += lpSum(X_allocation*cost_matrix) <= B, "Budget constraint" 

In [10]:
## Constraint 2: 4 adjacent cells 
### SUM Yij (i e Ij)<= 4*Xj for all j  !!! change to cardinality of Aj number of parcels adjacent to j 
count = [] 
c = 0 
for j in range(n_parcels):
    for k in range(len(adj2)): 
        if adj2[k] == parcel[j]:
            count.append(k) 
            c += 1 
            #print(count)
            #print(adj2[k])
            #print(parcel[j])
    if count != []:
        #print(lpSum(Yij_allocation[count[i]][count[i]] for i in range(len(count)) ) <= c*X_allocation[j])
        model += lpSum(Yij_allocation[count[i]][count[i]] for i in range(len(count))) <= c*X_allocation[j], "Max connection to j" + str(j)
    count = []
    c = 0 


In [11]:
model

Lab4:
MAXIMIZE
None
SUBJECT TO
Budget_constraint: 337671 X_1 + 38751 X_100 + 6822 X_101 + 22859 X_102
 + 177000 X_103 + 46342 X_104 + 266826 X_105 + 119105 X_106 + 896336 X_107
 + 257229 X_108 + 44718 X_109 + 5436 X_11 + 19224 X_110 + 25747 X_111
 + 26763 X_112 + 68715 X_113 + 416481 X_114 + 40642 X_115 + 100080 X_116
 + 74920 X_117 + 70705 X_118 + 20598 X_119 + 182715 X_12 + 309145 X_120
 + 152280 X_13 + 37921 X_14 + 107408 X_15 + 204360 X_16 + 180330 X_17
 + 163980 X_18 + 25005 X_19 + 19860 X_2 + 213367 X_20 + 46555 X_21
 + 15025 X_22 + 43240 X_23 + 16490 X_24 + 849971 X_25 + 12635 X_26
 + 40166 X_27 + 47970 X_28 + 19350 X_29 + 48252 X_3 + 28832 X_30 + 104752 X_31
 + 98049 X_32 + 22824 X_33 + 127848 X_34 + 155445 X_35 + 48420 X_36
 + 113676 X_37 + 26160 X_38 + 11070 X_39 + 80685 X_4 + 13920 X_40 + 25335 X_41
 + 19899 X_42 + 837704 X_43 + 128910 X_44 + 497513 X_45 + 118480 X_46
 + 90828 X_47 + 72005 X_48 + 253632 X_49 + 49008 X_5 + 235680 X_50
 + 45305 X_51 + 72344 X_52 + 112566 X_53 

In [12]:
## Constraint 3: Max 1 connection from i
#### SUM Yij (j e Ii) <= Xi 

count = [] 
for i in range(n_parcels):
    for k in range(len(adj1)): 
        if adj1[k] == parcel[i]:
            count.append(k) 
            #print(count)
            #print(adj2[k])
            #print(parcel[j])
    if count != []:
        #print(lpSum(Yij_allocation[count[i]][count[i]] for i in range(len(count)) ) <= X_allocation[i])
        model += lpSum(Yij_allocation[count[j]][count[j]] for j in range(len(count))) <= X_allocation[i], "Max connection from i" + str(i)
    count = []


In [13]:
## Constraint 4: Number of arcs must be less or equal to the number of nodes 
model += lpSum(Yij_allocation[j][j] for j in range(len(Yij_allocation))) == lpSum(X_allocation[i] for i in range(0,120))-1, "Number of arcs" 

In [14]:
## Constraint 5: Number of arcs must be less or equal to the number of nodes 
### Zij >= Wi + 1 - m(1-Yij) for all i, j adjacent, i != j 

for i in range(n_parcels):
    for k in range(len(adj1)):
        if adj1[k] == i :
            count.append(k)
    if count != []:
        for j in range(len(count)):
            Zij_allocation[count[j]] >= W_allocation[i-1] + 1 - m*(1-Yij_allocation[count[j]][count[j]])
            model += Zij_allocation[count[j]] >= W_allocation[i-1] + 1 - m*(1-Yij_allocation[count[j]][count[j]]), "Tail function" + str(i) + '_'+ str(j)
    count = []


In [15]:
## Constraint 6: Cycle breaking constraint 
### Wj >= SUM (ieIj) Zij for all j 

count = []
for i in range(n_parcels):
    for j in range(len(adj2)):
        if adj2[j] == i: 
            count.append(j)
    if count != []:
        W_allocation[i-1] == lpSum(Zij_allocation[count[k]] for k in range(len(count)) )
        model += W_allocation[i-1] == lpSum(Zij_allocation[count[k]] for k in range(len(count)) ), "Loop_break function" + str(i)
    count = []


In [16]:
## Consraint 7: Ensure that zij is minimal 
### Zij <= m*Yij 
count = []
ab = 0 
for i in range(n_parcels): 
    for j in range(len(adj2)): 
        if adj2[j] == i: 
            count.append(j)
    if count != []: 
        for k in range(len(count)): 
            Zij_allocation[count[k]] <= m*Yij_allocation[count[k]][count[k]]
            ab = ab+1
            model += Zij_allocation[count[k]] <= m*Yij_allocation[count[k]][count[k]], "Ensure_min_Zij" +  str(ab)
            
    count = []

In [17]:
## Constraint 8: Ensure that zij is minimal 
### Zij <= Wi+1 
count = []
ab = 0 
a = 0 
for i in range(n_parcels): 
    for j in range(len(adj1)): 
        if adj1[j] == i: 
            count.append(j)
    if count != []:
        a = a + 1
        for k in range(len(count)): 
            Zij_allocation[count[k]] <= W_allocation[i-1]+1
            ab = ab+1
            model += Zij_allocation[count[k]] <= W_allocation[i-1]+1, "Ensure_min_Zij_2" + str(ab)
            
    count = []

In [18]:
#core parcel : parcel 10, 32.031, 0.00
model += X_allocation[22] == 1, "Core parcel"

In [19]:
# Objective function 

obj_func = lpSum(X_allocation*area_matrix)
model += obj_func
#print(model)



In [20]:
print(model)

Lab4:
MAXIMIZE
37.519*X_1 + 32.031*X_10 + 12.917*X_100 + 2.274*X_101 + 22.859*X_102 + 11.8*X_103 + 23.171*X_104 + 19.059*X_105 + 23.821*X_106 + 56.021*X_107 + 36.747*X_108 + 14.906*X_109 + 1.359*X_11 + 19.224*X_110 + 25.747*X_111 + 26.763*X_112 + 22.905*X_113 + 32.037*X_114 + 20.321*X_115 + 11.12*X_116 + 18.73*X_117 + 14.141*X_118 + 6.866*X_119 + 12.181*X_12 + 18.185*X_120 + 25.38*X_13 + 37.921*X_14 + 7.672*X_15 + 34.06*X_16 + 12.022*X_17 + 10.932*X_18 + 5.001*X_19 + 6.62*X_2 + 30.481*X_20 + 9.311*X_21 + 3.005*X_22 + 8.648*X_23 + 3.298*X_24 + 20.731*X_25 + 2.527*X_26 + 5.738*X_27 + 3.198*X_28 + 6.45*X_29 + 24.126*X_3 + 7.208*X_30 + 26.188*X_31 + 14.007*X_32 + 11.412*X_33 + 18.264*X_34 + 10.363*X_35 + 9.684*X_36 + 18.946*X_37 + 13.08*X_38 + 1.845*X_39 + 5.379*X_4 + 1.74*X_40 + 1.689*X_41 + 6.633*X_42 + 59.836*X_43 + 42.97*X_44 + 21.631*X_45 + 29.62*X_46 + 30.276*X_47 + 14.401*X_48 + 42.272*X_49 + 12.252*X_5 + 58.92*X_50 + 9.061*X_51 + 18.086*X_52 + 18.761*X_53 + 4.151*X_54 + 16.556*X_55

In [21]:
#Save code into a lp file 
model.writeLP("model_lab4_test.lp")


[W_10,
 W_100,
 W_12,
 W_13,
 W_14,
 W_15,
 W_16,
 W_17,
 W_18,
 W_19,
 W_2,
 W_20,
 W_21,
 W_22,
 W_23,
 W_24,
 W_25,
 W_26,
 W_27,
 W_28,
 W_29,
 W_3,
 W_31,
 W_32,
 W_33,
 W_34,
 W_35,
 W_36,
 W_37,
 W_38,
 W_39,
 W_4,
 W_40,
 W_42,
 W_45,
 W_46,
 W_47,
 W_48,
 W_49,
 W_5,
 W_50,
 W_51,
 W_53,
 W_54,
 W_55,
 W_56,
 W_57,
 W_58,
 W_59,
 W_6,
 W_60,
 W_62,
 W_63,
 W_64,
 W_65,
 W_66,
 W_67,
 W_68,
 W_7,
 W_70,
 W_71,
 W_72,
 W_73,
 W_74,
 W_75,
 W_76,
 W_77,
 W_78,
 W_79,
 W_8,
 W_80,
 W_81,
 W_82,
 W_83,
 W_84,
 W_85,
 W_86,
 W_87,
 W_88,
 W_89,
 W_90,
 W_91,
 W_92,
 W_93,
 W_94,
 W_96,
 W_97,
 W_98,
 W_99,
 X_1,
 X_10,
 X_100,
 X_101,
 X_102,
 X_103,
 X_104,
 X_105,
 X_106,
 X_107,
 X_108,
 X_109,
 X_11,
 X_110,
 X_111,
 X_112,
 X_113,
 X_114,
 X_115,
 X_116,
 X_117,
 X_118,
 X_119,
 X_12,
 X_120,
 X_13,
 X_14,
 X_15,
 X_16,
 X_17,
 X_18,
 X_19,
 X_2,
 X_20,
 X_21,
 X_22,
 X_23,
 X_24,
 X_25,
 X_26,
 X_27,
 X_28,
 X_29,
 X_3,
 X_30,
 X_31,
 X_32,
 X_33,
 X_34,
 X_35,
 X_36,
 X_37,
 

In [ ]:
#play around with CBC solver 
model.solve(PULP_CBC_CMD())

status =  LpStatus[model.status]

print(status)